In [18]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

In [19]:
# Carregar dados
base = pd.read_csv('/home/Ront/Documentos/Card-14/Pratica/Seção7/games.csv')
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Remover valores nulos
base = base.dropna()
base.shape[0]

6825

In [21]:
# Preparar dados de entrada e saída
X = base.iloc[:, [1, 2, 3, 4, 10, 11, 13, 14, 15]].values 

In [22]:
# Saídas: vendas em América do Norte, Europa e Japão
y_na = base.iloc[:, 5].values
y_eu = base.iloc[:, 6].values
y_jp = base.iloc[:, 7].values

In [23]:
# Codificar variáveis categóricas
onehotencoder = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(), [0, 1, 2, 3, 7, 8])],
    remainder='passthrough'
)
X = onehotencoder.fit_transform(X).toarray()

In [24]:
X.shape

(6825, 1615)

In [25]:
# Criar modelo com API Funcional (múltiplas saídas)
camada_entrada = Input(shape=(X.shape[1],))
camada_oculta1 = Dense(units=500, activation='relu')(camada_entrada)
camada_oculta2 = Dense(units=500, activation='relu')(camada_oculta1)

In [26]:
# Saídas separadas para cada região
camada_saida_na = Dense(units=1, activation='linear', name='na')(camada_oculta2)
camada_saida_eu = Dense(units=1, activation='linear', name='eu')(camada_oculta2)
camada_saida_jp = Dense(units=1, activation='linear', name='jp')(camada_oculta2)

regressor = Model(inputs=camada_entrada, outputs=[camada_saida_na, camada_saida_eu, camada_saida_jp])

In [27]:
# Compilar modelo
regressor.compile(
    optimizer='adam',
    loss={'na': 'mae', 'eu': 'mae', 'jp': 'mae'},
    metrics={'na': 'mae', 'eu': 'mae', 'jp': 'mae'}
)

In [15]:
# Treinar modelo
regressor.fit(
    X, 
    {'na': y_na, 'eu': y_eu, 'jp': y_jp},
    batch_size=100,
    epochs=100
)

Epoch 1/100


2026-02-13 15:16:31.688997: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 44089500 exceeds 10% of free system memory.


69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - eu_loss: 2.1938 - eu_mae: 2.2105 - jp_loss: 3.5026 - jp_mae: 3.5346 - loss: 6.9001 - na_loss: 1.1453 - na_mae: 1.1551 
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - eu_loss: 0.5122 - eu_mae: 0.5132 - jp_loss: 0.5793 - jp_mae: 0.5831 - loss: 2.2113 - na_loss: 1.1116 - na_mae: 1.1150
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - eu_loss: 0.4684 - eu_mae: 0.4724 - jp_loss: 0.2618 - jp_mae: 0.2641 - loss: 1.1886 - na_loss: 0.4499 - na_mae: 0.4521
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - eu_loss: 0.3311 - eu_mae: 0.3317 - jp_loss: 0.2987 - jp_mae: 0.2986 - loss: 1.0644 - na_loss: 0.4325 - na_mae: 0.4341
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - eu_loss: 0.5101 - eu_mae: 0.5088 - jp_loss: 0.2344 - jp_mae: 0.2328 - loss: 1.1841 - na_loss: 0.4412 - na_mae: 0.4425
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - eu_loss: 0.4618 - eu_mae: 0.4655 - jp_loss: 0.1713 - jp_mae: 0.1723 - loss: 1.0053 - na_loss: 0.3

In [28]:
# Fazer previsões
previsao_na, previsao_eu, previsao_jp = regressor.predict(X)

 59/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2026-02-13 15:19:05.669843: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 44089500 exceeds 10% of free system memory.


214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [29]:
# Visualizar algumas previsões
resultados = pd.DataFrame({
    'NA_Real': y_na[:10],
    'NA_Prev': previsao_na[:10].flatten(),
    'EU_Real': y_eu[:10],
    'EU_Prev': previsao_eu[:10].flatten(),
    'JP_Real': y_jp[:10],
    'JP_Prev': previsao_jp[:10].flatten()
})
resultados

,NA_Real,NA_Prev,EU_Real,EU_Prev,JP_Real,JP_Prev
0,41.36,4.936965,28.96,-4.652270,3.77,-10.070376
1,15.68,9.463717,12.76,-11.462957,3.79,-21.898102
2,15.61,3.702686,10.93,-1.450261,3.28,-7.766464
3,11.28,6.477405,9.14,-6.407278,6.50,-13.565803
4,13.96,2.604191,9.18,-1.149464,2.93,-4.665582
5,14.44,8.461782,6.94,-9.287666,4.70,-18.932350
6,9.71,6.757654,7.47,-6.957793,4.13,-14.382982
7,8.92,3.316145,8.03,-0.726446,3.60,-5.838076
8,15.00,2.410751,4.89,-0.632133,0.24,-4.157235
9,9.01,2.040057,8.49,0.810516,2.53,-2.201727
